In [1]:
import requests 
import yaml
import json
import datetime as dt
import time
import os
import sys
import random
from azure.storage.blob import BlobServiceClient, BlobType
from email_validator import validate_email, EmailNotValidError
from itertools import chain

# pathing 
PARENT_DIR = os.path.abspath(os.path.pardir)
module_path = os.path.abspath(os.path.join(PARENT_DIR)) # for other functions
if module_path not in sys.path:
  sys.path.append(module_path)
from funcs.utils import log_azure, request, load_json, clean_field_text, load_config, get_email, send_email, create_url
from funcs.funcs import combine_qa_keys, get_sm_survey_responses, process_sm_responses, post_cos

DATA_DIR = os.path.join(PARENT_DIR, "data")

# creds
with open(os.path.join(PARENT_DIR, "creds", "api-key.yaml"), "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = data['sm']['real']
# CareerOneStop Survey 
COS_DATA = data['cos']


---

##### **Main Functions** 

* `get_qa_key()` - GET (or load cached copy) of question/answer key from SM or COS 

<br>

* `combine_qa_keys()` - Combine the SM and COS question/answer keys into one combined key/translation map between the APIs
    - Generates a refreshed map if a change is detected in the SM survey or the COS survey. 
        - The COS survey should not change at all. 
        - The survey monkey questions which match to the COS survey questions should not change (they are intended to just be a port).

<br>

* `get_sm_responses()` - GET SM survey responses 

<br>

* `process_sm_responses()` - filter and process new SM survey responses from get_sm_responses()
    - Checks against DB for already processed responses 
    - Checks for unexpected question ids vs. the combined Q/A key.
        - Attempts to refresh the combined Q/A key if any unexpected question ids are found.
    - Adds matching information from the combined Q/A key to the survey responses   
    - Loads new responses into database (into 'processing' table) until they are finished

<br>

* `post_cos()` 
    - Creates COS JSON request objects from each SM survey response object 
        - If a SM survey response is missing an answer for a skills-survey question, it fills the corresponding question in the COS object with an answer of "Beginner"
    -  POSTS each request object to the COS Skills Matcher API 
    - Stores the COS response alongside the original SM survey response, updating the database 
    
<br>

* `send_email()` - Send email if respondent provided valid email address.   

In [4]:
# ## DEMO 10/16
# # sm_survey_responses = get_sm_survey_responses()
# with open('temp_sm_survey_responses_cache_test.json', 'r') as file: 
#     sm_survey_responses = json.load(file)
    
# processed_sm_responses = process_sm_responses(sm_survey_responses)
# processed_sm_cos_responses = post_cos(processed_sm_responses)
# send_email(cos_recommendations[0])

with open('temp_processed_sm_cos_responses_cache_test.json', 'r') as file: 
    processed_sm_cos_responses = json.load(file)

In [5]:
processed_sm_responses[1]['cos_response']

{'SKARankList': [{'OnetCode': '25-1062.00',
   'Score': 0.774774582058987,
   'Rank': 1,
   'Outlook': 'Bright',
   'AnnualWages': 80910.0,
   'TypicalEducation': 'Doctoral or professional degree',
   'OccupationTitle': 'Area, Ethnic, and Cultural Studies Teachers, Postsecondary',
   'EduCode': 2.0},
  {'OnetCode': '25-1126.00',
   'Score': 0.7716870680368859,
   'Rank': 2,
   'Outlook': 'Bright',
   'AnnualWages': 78780.0,
   'TypicalEducation': 'Doctoral or professional degree',
   'OccupationTitle': 'Philosophy and Religion Teachers, Postsecondary',
   'EduCode': 2.0},
  {'OnetCode': '25-1124.00',
   'Score': 0.76988846855684,
   'Rank': 3,
   'Outlook': 'Bright',
   'AnnualWages': 76030.0,
   'TypicalEducation': 'Doctoral or professional degree',
   'OccupationTitle': 'Foreign Language and Literature Teachers, Postsecondary',
   'EduCode': 2.0},
  {'OnetCode': '25-1065.00',
   'Score': 0.765700153770811,
   'Rank': 4,
   'Outlook': 'Bright',
   'AnnualWages': 83770.0,
   'TypicalEd

In [18]:

response_1 = processed_sm_cos_responses[1].copy()

with open('../creds/api-key.yaml', 'r') as file: 
    data = yaml.full_load(file)['email']['shared-dil-account']

APP_PASSWORD = data['app-password']
API_KEY = data['ee-api-key']
SENDER_EMAIL = data['sender-email']
RECEIVER_EMAIL = get_email(response_1)

# general email settings 
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_SUBJECT = f'Your survey result for {dt.datetime.now().strftime("%B %d, %Y")}'

rec_list = []
for rec in response_1['cos_response']['SKARankList']:
    cleaned_rec = {}
    cleaned_rec['Your Match Rank'] = rec['Rank'] 
    cleaned_rec['Job Title'] = rec['OccupationTitle']
    cleaned_rec['Typical Wages (Annual)'] = f"${rec['AnnualWages']:,.0f}"
    cleaned_rec['Typical Education'] = rec['TypicalEducation']
    # Create url (not embedded)
    cleaned_rec['Link'] = create_url(job_title=rec['OccupationTitle'], 
                                     onet_code=rec['OnetCode'])
    rec_list.append(cleaned_rec)

## Format/Style HTML table 
table_headers = rec_list[0].keys()
table_html = "<table>"
table_html += "<tr>"

# Header Style
header_style = "font-weight: bold; font-size: 20px;"
for header in table_headers:
    table_html += f"<th style='{header_style}'>{header}</th>"
table_html += "</tr>"

# Cell Style
cell_style = "font-weight: normal; font-size: 16px;"  
for rec in rec_list:
    table_html += "<tr>"
    for header in table_headers:
        table_html += f"<td style='{cell_style}'>{rec[header]}</td>"
    table_html += "</tr>"
table_html += "</table>"


In [32]:
message_style = "font-weight: bold; font-style: italic; font-size: 16px;"

with open("../funcs/email_message_text.txt", "r") as file: 
    message_text = file.read()

message_text = message_text.replace('\n','<br>')
# section_separator = "<P style=\"page-break-before: always\">"
section_separator = "<br><hr><br>"


message = f"<div style=\"{message_style}\">{message_text}{section_separator}</div>{table_html}"

from bs4 import BeautifulSoup

soup = BeautifulSoup(message, 'html.parser')
soup

<div style="font-weight: bold; font-style: italic; font-size: 16px;">Thank you for participating in our workforce survey about community member job skills, experiences, and interests. Your inputs will be extremely important as we seek to understand the employment strengths and desires of people from across the state, to make informed recommendations to workforce decision makers about how to improve access to employment for ALL Delawareans. <br/><br/>As promised, below are the results from the skills assessment portion of the survey. We have included recommendations of careers that you are well-suited for, based on your responses. You can click on each career to learn more! <br/><br/>We also wanted to share this informational flyer with important links to different workforce agencies and resources that may be useful to you in the future: https://drive.google.com/file/d/1fGdys5Z4WbkEZ_Y5JSStqPGDcWt6YWBs/view<br/><br/>Finally, we will be sending you your $10 gift card for participating in

In [ ]:
## TO-DO: Loop over the question_answer key and not the provided responses, avoid having to auto-fill again at the end

# processed_responses = []
# placeholder_processed_response_ids = []
# combined_map = combine_qa_keys()

# for resp in sm_survey_responses['data']:      
#     if resp['id'] not in placeholder_processed_response_ids:
#         # TO-DO: Do the email validation in the actual email sending function 
#         resp_dict = {
#         'response_id':resp['id'],
#         'collector_id':resp['collector_id'], 
#         'questions':[] 
#         }
#         resp_question_answers = {q['id']:q['answers'] for p in resp['pages'] for q in p['questions']}

#         ## Add questions information from combined qa key 
#         for q_map in  list(combined_map['non-skills-matcher'].values()) +  list(combined_map['skills-matcher'].values()):
#             # Fill-in omitted questions from key
#             if q_map['question_id']['sm'] not in resp_question_answers.keys():
#                 q_map['auto_filled'] = True
#                 # Auto-fill beginner level answer for skills-matcher questions
#                 q_map['answers'] = [q_map['answers'][0]] if q_map['question_type'] == 'skills-matcher' else None
#                 resp_dict['questions'].append(q_map)

#             elif q_map['answers'] is not None: # if the answer key has answer choices listed for the question
                
#                 q_map_answer_key = {a['id']['sm']:a for a in q_map['answers']}
#                 try: 
#                     resp_answers = [q_map_answer_key[a['choice_id']]
#                         if 'choice_id' in a.keys() else {'id':{'sm':a['other_id']}, 'text':{'sm':a['text']}}
#                         for a in resp_question_answers[q_map['question_id']['sm']]]
#                 except: 
#                     print(resp_question_answers[q_map['question_id']['sm']])
#                     print(q_map_answer_key)
#                 q_map['answers'] = resp_answers

#             else: 
#                 q_map['answers'] = resp_question_answers[q_map['question_id']['sm']]
                
#             resp_dict['questions'].append(q_map)

#     processed_responses.append(resp_dict)       

# processed_responses